## Accenture - Supply Chain


In [551]:
# Packaging management
import numpy as np
import pandas as pd
import csv 
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from statsmodels.formula.api import ols
import seaborn as sns

# Data Preparation

In [552]:
## Distance matrix preparation

## Read raw data from csv files
pd_cities_raw = pd.read_csv("datas/cities_data.csv" ,sep=";")

city_list_s = pd_cities_raw['city_from_name'].unique()
city_list_d = pd_cities_raw['city_to_name'].unique()
all_cities= list(set(city_list_s) | set(city_list_d))

city_dict = {all_cities[i]:i for i in range(0,len(all_cities))}
n_cities = len(all_cities)

distance_mat = [[0]*n_cities]*n_cities


for index, row in pd_cities_raw.iterrows():
    s = city_dict[row["city_from_name"]]
    d = city_dict[row["city_to_name"]]
    distance = row["distance"]
    distance_mat[s][d] = distance

In [553]:
## Product Matrix

pd_products_raw = pd.read_csv("datas/product_attributes.csv", sep=",")


##dataframe quering example  !(return a string value)
# pd_products_raw.loc[pd_products_raw["product_id"]==1672707]["material_handling"].to_string(index=False)


In [554]:
## Probability dictionary calculation
pd_orders_raw=pd.read_csv("datas/orders.csv",sep=";")

pd_orders_raw.loc[pd_orders_raw["origin_port"] == "BCN", "origin_port"] = "Barcelona"
pd_orders_raw.loc[pd_orders_raw["origin_port"] == "ATHENAS", "origin_port"] = "Athens"


cities_origin = pd_orders_raw['origin_port'].unique()
cities_hub = pd_orders_raw['logistic_hub'].unique()
cities_customer =  pd_orders_raw['customer'].unique()
pd_origin_count = pd_orders_raw.groupby(['origin_port', 'late_order']).size()
pd_hub_count = pd_orders_raw.groupby(['logistic_hub', 'late_order']).size()
pd_customer_count = pd_orders_raw.groupby(['customer', 'late_order']).size()

city_count_origin =  pd_orders_raw.groupby(['origin_port']).size()
city_count_hub = pd_orders_raw.groupby(['logistic_hub']).size()
city_count_customer = pd_orders_raw.groupby(['customer']).size()

delay_rate_origin = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_hub = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_customer = {all_cities[i]:0 for i in range(0,len(all_cities))}


origin_count_dict =  pd_orders_raw.groupby(['origin_port','late_order']).size().to_dict()
hub_count_dict =  pd_orders_raw.groupby(['logistic_hub','late_order']).size().to_dict()
customer_count_dict =  pd_orders_raw.groupby(['customer','late_order']).size().to_dict()

In [555]:
for key, values in origin_count_dict:
    total = city_count_origin[key]
    if (values == True):
        true_count_origin = origin_count_dict[key,values]
        delay_rate_origin[key] = true_count_origin/total
print("origin city")
print(delay_rate_origin)
for key, values in hub_count_dict:
    total = city_count_hub[key]
    if (values == True):
        true_count_origin = hub_count_dict[key,values]
        delay_rate_hub[key] = true_count_origin/total
print("hub city")
print(delay_rate_hub)
for key, values in customer_count_dict:
    total = city_count_customer[key]
    if (values == True):
        true_count_origin = customer_count_dict[key,values]
        delay_rate_customer[key] = true_count_origin/total
print("destination city")
print(delay_rate_customer)

origin city
{'Barcelona': 0.2702263870848024, 'Turin': 0, 'Vienna': 0, 'Zaragoza': 0, 'Berlin': 0, 'Cologne': 0, 'Marseille': 0, 'Bremen': 0, 'Helsinki': 0, 'Rotterdam': 0.16921047383841328, 'Amsterdam': 0, 'Lyon': 0, 'Copenhagen': 0, 'Rome': 0, 'Porto': 0, 'Dusseldorf': 0, 'Paris': 0, 'Prague': 0, 'Hanover': 0, 'Bratislava': 0, 'Hamburg': 0, 'Munich': 0, 'Budapest': 0, 'Bordeaux': 0, 'Warsaw': 0, 'Athens': 0.3741508954193759, 'Venlo': 0, 'Stockholm': 0, 'Valencia': 0, 'Lille': 0, 'Milan': 0, 'Madrid': 0, 'Lisbon': 0, 'Bucharest': 0, 'Naples': 0, 'Liege': 0, 'Malmö': 0}
hub city
{'Barcelona': 0, 'Turin': 0, 'Vienna': 0, 'Zaragoza': 0.3497337053005326, 'Berlin': 0, 'Cologne': 0, 'Marseille': 0, 'Bremen': 0, 'Helsinki': 0, 'Rotterdam': 0, 'Amsterdam': 0, 'Lyon': 0, 'Copenhagen': 0, 'Rome': 0.2575441412520064, 'Porto': 0, 'Dusseldorf': 0.18381530595941845, 'Paris': 0, 'Prague': 0, 'Hanover': 0, 'Bratislava': 0.3308839902026275, 'Hamburg': 0.20204841713221602, 'Munich': 0, 'Budapest': 0, '

In [556]:
threepl_list = pd_orders_raw['3pl'].unique()
procedures = pd_orders_raw['customs_procedures'].unique()

n3pl = len(threepl_list)
nprocedures= len(procedures)

dict_3pl = {threepl_list[i]:i for i in range(0,len(threepl_list))}
dict_procedures = {procedures[i]:i for i in range(0,len(procedures))}

pd_orders_raw['3pl'] = pd_orders_raw['3pl'].factorize()[0]
pd_orders_raw['customs_procedures'] = pd_orders_raw['customs_procedures'].factorize()[0]

pd_orders_raw

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order
0,366c7a3d298f,Rotterdam,0,0,Venlo,Marseille,1692723,583,True
1,45f906331e10,Rotterdam,1,1,Rome,Marseille,1644308,459,False
2,ac84a6e4af0f,Athens,0,1,Venlo,Paris,1684170,464,True
3,f5e98cb29790,Rotterdam,1,1,Lille,Milan,1620510,678,False
4,a9e7c9bee35b,Barcelona,0,1,Venlo,Berlin,1699372,353,False
...,...,...,...,...,...,...,...,...,...
114271,3f4b15fb770e,Rotterdam,0,1,Dusseldorf,Bordeaux,1681376,645,False
114272,d2e6978a38fd,Barcelona,1,2,Dusseldorf,Berlin,1676942,502,False
114273,b88babd5c7bd,Rotterdam,0,0,Dusseldorf,Rome,1692737,464,False
114274,b0b5c761613f,Barcelona,3,2,Dusseldorf,Munich,1699974,388,False


In [557]:
p_origin = pd_orders_raw['origin_port'].tolist()
p_hub = pd_orders_raw['logistic_hub'].tolist()
p_customer =  pd_orders_raw['customer'].tolist()

i=0
for item in p_origin:
    p_origin[i] = delay_rate_origin[item]
    i+=1

i=0
for item in p_hub:
    if type(item) != str:
        p_hub[i] = 0
        i+=1
    else: 
        p_hub[i] = delay_rate_hub[item]
        i+=1
        
i=0
for item in p_customer:
    p_customer[i] = delay_rate_customer[item]
    i+=1

In [558]:
pd_orders_raw

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order
0,366c7a3d298f,Rotterdam,0,0,Venlo,Marseille,1692723,583,True
1,45f906331e10,Rotterdam,1,1,Rome,Marseille,1644308,459,False
2,ac84a6e4af0f,Athens,0,1,Venlo,Paris,1684170,464,True
3,f5e98cb29790,Rotterdam,1,1,Lille,Milan,1620510,678,False
4,a9e7c9bee35b,Barcelona,0,1,Venlo,Berlin,1699372,353,False
...,...,...,...,...,...,...,...,...,...
114271,3f4b15fb770e,Rotterdam,0,1,Dusseldorf,Bordeaux,1681376,645,False
114272,d2e6978a38fd,Barcelona,1,2,Dusseldorf,Berlin,1676942,502,False
114273,b88babd5c7bd,Rotterdam,0,0,Dusseldorf,Rome,1692737,464,False
114274,b0b5c761613f,Barcelona,3,2,Dusseldorf,Munich,1699974,388,False


In [559]:
pd_orders_raw["p_origin"] = p_origin
pd_orders_raw["p_hub"] = p_hub
pd_orders_raw["p_customer"] = p_customer
pd_orders_raw

origins = pd_orders_raw['origin_port'].tolist()
hubs = pd_orders_raw['logistic_hub'].tolist()
customers =  pd_orders_raw['customer'].tolist()

total_distance = []
for i in range(len(origins)):
   if type(hubs[i]) != str:
       total_distance.append(distance_mat[city_dict[origins[i]]][city_dict[customers[i]]])
   else:
        a = distance_mat[city_dict[origins[i]]][city_dict[hubs[i]]]
        b = distance_mat[city_dict[hubs[i]]][city_dict[customers[i]]]
        total_distance.append(a+b)
        
pd_orders_raw["total_d"] = total_distance

In [560]:
pd_products_raw.loc[len(pd_products_raw.index)] = [-1, 0, 6] 
pd_final = pd.DataFrame()
pd_final = pd.merge(pd_orders_raw, pd_products_raw, on="product_id")
pd_final["late_order"] = pd_final["late_order"].astype(int)

pd_final = pd_final.rename(columns = {"3pl":"threepl"})

# Data Test

# Correlation 

# 1: Combining "product attributes" & "orders"
attribute = pd.read_excel("datas/product_attributes.csv")
orders = pd.read_excel("datas/orders.csv")  
# merging the files
combined = attribute[["product_id"]].merge(orders[["product_id"]], 
                                     on = "product_id", 
                                     how = "left")
# creating a new file
combined.to_csv ("combined.csv", 
                  index = None,
                  header=True)

# 2: Creating correlation formula


# Data Training

In [561]:
pd_final.dtypes


order_id               object
origin_port            object
threepl                 int64
customs_procedures      int64
logistic_hub           object
customer               object
product_id              int64
units                   int64
late_order              int64
p_origin              float64
p_hub                 float64
p_customer            float64
total_d               float64
weight                  int64
material_handling       int64
dtype: object

In [562]:
pd_final = pd_final.drop(['product_id','order_id','origin_port', 'logistic_hub','customer'],axis=1)

In [563]:
# column transformation
categorical_columns = ["threepl","customs_procedures","material_handling"]
numerical_columns = ["units","p_origin","p_hub","p_customer","total_d","weight"]

# determine X
X = pd_final[categorical_columns + numerical_columns]
y = pd_final["late_order"]

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.3
)

# categorical pipeline
categorical_pipe = Pipeline([("ordinar", OrdinalEncoder())])

# numerical pipeline
numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean")),])

# aggregating both the pipeline
preprocessing = ColumnTransformer(
    [
        ("cat", categorical_pipe, categorical_columns),
        ("num", numerical_pipe, numerical_columns),
    ]
)


rf = Pipeline(
    [("preprocess", preprocessing), ("classifier", linear_model.LinearRegression())]
)

# train
rf.fit(X_train, y_train)

# predict
predict = rf.predict(X_test)
X = pd.get_dummies(data=pd_final, drop_first=True)
X.head()

,threepl,customs_procedures,units,late_order,p_origin,p_hub,p_customer,total_d,weight,material_handling
0,0,0,583,1,0.169210,0.187384,0.327694,1091.6809,1778,5
1,1,2,385,0,0.270226,0.202048,0.203826,1211.7994,1778,5
2,1,1,484,0,0.169210,0.183815,0.195640,234.7418,1778,5
3,1,0,408,0,0.169210,0.257544,0.183145,1473.7563,1778,5
4,1,0,526,0,0.374151,0.359582,0.227625,1975.3037,1778,5


In [564]:
model = rf[1]
model.coef_

import statsmodels.api as sm
X_train_Sm= sm.add_constant(X_train)
ls=sm.OLS(y_train,X_train_Sm).fit()
print(ls.summary())

                            OLS Regression Results                            
Dep. Variable:             late_order   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     1123.
Date:                Sat, 12 Nov 2022   Prob (F-statistic):               0.00
Time:                        21:02:29   Log-Likelihood:                -40375.
No. Observations:               79993   AIC:                         8.077e+04
Df Residuals:                   79983   BIC:                         8.086e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.0117      0

In [565]:
test = pd.read_csv("datas/test.csv",sep=";")
orders = test["order_id"].to_list()

In [566]:
test

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units
0,0e364fa5c795,Rotterdam,v_002,CRF,Warsaw,Munich,1687644,477
1,3ef49bd5a55b,Barcelona,v_002,CRF,Venlo,Berlin,1699689,240
2,9ab5b9685bd5,Rotterdam,v_004,DTD,Bratislava,Paris,1652615,506
3,bfce5b4fc4fa,Rotterdam,v_002,DTD,Rome,Hanover,1669740,553
4,d94453ec8ec5,Barcelona,v_001,CRF,Bratislava,Copenhagen,1695138,482
...,...,...,...,...,...,...,...,...
28558,d268acf6459e,Athens,v_004,CRF,Dusseldorf,Naples,1657474,496
28559,1aefc30b0eb3,Rotterdam,v_004,DTD,Hamburg,Bordeaux,1657437,577
28560,646a2e50e170,Rotterdam,v_001,CRF,Venlo,Milan,1689781,483
28561,bf5177549be9,ATHENAS,v_001,CRF,Lille,Milan,1672088,564


In [571]:

test.loc[test["origin_port"] == "BCN", "origin_port"] = "Barcelona"
test.loc[test["origin_port"] == "ATHENAS", "origin_port"] = "Athens"

cities_origin = test['origin_port'].to_list()
cities_hub = test['logistic_hub'].to_list()
cities_customer =  test['customer'].to_list()

p_origin = []
p_hub = []
p_customer = []

for item in cities_origin:
    p_origin.append(delay_rate_origin[item])

for item in cities_hub:
    if type(item) != str:
        p_hub.append(0)
    else:
        p_hub.append(delay_rate_hub[item])
    
for item in cities_customer:
    p_customer.append(delay_rate_customer[item])

In [572]:
test

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,p_origin,p_hub,p_customer
0,0e364fa5c795,Rotterdam,0,0,Warsaw,Munich,1687644,477,0.169210,0.227552,0.194146
1,3ef49bd5a55b,Barcelona,0,0,Venlo,Berlin,1699689,240,0.270226,0.187384,0.195845
2,9ab5b9685bd5,Rotterdam,1,1,Bratislava,Paris,1652615,506,0.169210,0.330884,0.307145
3,bfce5b4fc4fa,Rotterdam,0,1,Rome,Hanover,1669740,553,0.169210,0.257544,0.187556
4,d94453ec8ec5,Barcelona,2,0,Bratislava,Copenhagen,1695138,482,0.270226,0.330884,0.266762
...,...,...,...,...,...,...,...,...,...,...,...
28558,d268acf6459e,Athens,1,0,Dusseldorf,Naples,1657474,496,0.374151,0.183815,0.215299
28559,1aefc30b0eb3,Rotterdam,1,1,Hamburg,Bordeaux,1657437,577,0.169210,0.202048,0.319556
28560,646a2e50e170,Rotterdam,2,0,Venlo,Milan,1689781,483,0.169210,0.187384,0.203826
28561,bf5177549be9,Athens,2,0,Lille,Milan,1672088,564,0.374151,0.359582,0.203826


In [574]:
test["p_origin"] = p_origin
test["p_hub"] = p_hub
test["p_customer"] = p_customer
#pd_orders_raw

origins = test['origin_port'].tolist()
hubs = test['logistic_hub'].tolist()
customers =  test['customer'].tolist()

total_distance = []
for i in range(len(origins)):
   if type(hubs[i]) != str:
       total_distance.append(distance_mat[city_dict[origins[i]]][city_dict[customers[i]]])
   else:
        a = distance_mat[city_dict[origins[i]]][city_dict[hubs[i]]]
        b = distance_mat[city_dict[hubs[i]]][city_dict[customers[i]]]
        total_distance.append(a+b)
        
test["total_d"] = total_distance

In [570]:
test

""


In [575]:
threepl_list = test['3pl'].unique()
procedures = test['customs_procedures'].unique()

n3pl = len(threepl_list)
nprocedures= len(procedures)

dict_3pl = {threepl_list[i]:i for i in range(0,len(threepl_list))}
dict_procedures = {procedures[i]:i for i in range(0,len(procedures))}

test['3pl'] = test['3pl'].factorize()[0]
test['customs_procedures'] = test['customs_procedures'].factorize()[0]

test["p_origin"] = p_origin
test["p_hub"] = p_hub
test["p_customer"] = p_customer

test_final = pd.DataFrame()
test_final = pd.merge(test, pd_products_raw, on="product_id")
test_final

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,p_origin,p_hub,p_customer,total_d,weight,material_handling
0,0e364fa5c795,Rotterdam,0,0,Warsaw,Munich,1687644,477,0.169210,0.227552,0.194146,1795.4857,1974,5
1,6c8c11251114,Rotterdam,0,0,Venlo,Barcelona,1687644,414,0.169210,0.187384,0.225946,1314.8781,1974,5
2,81d3be5e1eff,Athens,0,1,Warsaw,Lyon,1687644,536,0.374151,0.227552,0.324902,1821.9048,1974,5
3,870a9b65a1b5,Rotterdam,2,0,Dusseldorf,Berlin,1687644,535,0.169210,0.183815,0.195845,789.4519,1974,5
4,eff072956d66,Rotterdam,0,0,Rome,Milan,1687644,266,0.169210,0.257544,0.203826,2066.5870,1974,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28558,5e5aac335f95,Rotterdam,1,0,Rome,Porto,1690006,520,0.169210,0.257544,0.317446,2827.2694,1375,5
28559,753fe21cc3cb,Athens,0,1,Rome,Milan,1697086,431,0.374151,0.257544,0.203826,2066.5870,2009,2
28560,9a4166ebe45d,Rotterdam,1,0,Dusseldorf,Porto,1652604,495,0.169210,0.183815,0.317446,1735.3404,547,4
28561,7115201bce91,Barcelona,3,2,Hamburg,Paris,1652604,434,0.270226,0.202048,0.307145,788.3384,547,4


In [576]:
test = test_final.drop(['product_id','order_id','origin_port', 'logistic_hub','customer'],axis=1)
model.predict(test_final)

/usr/local/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


ValueError: X has 7 features, but LinearRegression is expecting 9 features as input.